<a href="https://colab.research.google.com/github/chandu8542/codemix/blob/main/sarc_self_match_bi_LSTM_fastext_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install wget

In [2]:
%tensorflow_version 2.x

In [3]:
#import wget
import os
import numpy as np
import pandas as pd

In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.utils import shuffle

import matplotlib.pyplot as plt

In [5]:

# import gzip
# import shutil
# from zipfile import ZipFile 

In [6]:
# from tensorflow.core.protobuf import rewriter_config_pb2
# tf.keras.backend.clear_session()

# config_proto = tf.compat.v1.ConfigProto()
# off = rewriter_config_pb2.RewriterConfig.OFF
# config_proto.graph_options.rewrite_options.arithmetic_optimization = off
# config_proto.graph_options.rewrite_options.memory_optimization = off

In [7]:
# tf.config.optimizer.set_experimental_options({'arithmetic_optimization': rewriter_config_pb2.RewriterConfig.OFF,
#                                               'memory_optimization': rewriter_config_pb2.RewriterConfig.OFF})


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'''def load_file(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text.split("\n")'''

In [ ]:
def get_max_len(tweets):
    max_tweet_len = len(max(tweets, key=len).split())
    return max_tweet_len

In [ ]:
def get_mean_len(tweets):
    sum_of_length = sum([len(l.split()) for l in tweets])
    avg_tweet_len = sum_of_length / float(len(tweets))
    return avg_tweet_len

In [ ]:
'''url_common = "https://www.dropbox.com/s/flh1fjynqvdsj4p/lexvec.commoncrawl.300d.W.pos.vectors.gz?dl=1"
url_wiki = "https://www.dropbox.com/s/kguufyc2xcdi8yk/lexvec.enwiki%2Bnewscrawl.300d.W.pos.vectors.gz?dl=1"

file_commmon = 'lexvec.commoncrawl.300d.W.pos.vectors.gz'
file_wiki = 'lexvec.enwiki+newscrawl.300d.W.pos.vectors.gz'

if os.path.isfile(file_wiki[:-3]):
    print ("File exist")
else:
    wget.download(url_wiki)  
    with gzip.open(file_wiki, 'rb') as f_in:
        with open(file_wiki[:-3], 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)'''

df_embedding = pd.read_csv('/content/drive/MyDrive/models/hing_emb', sep=" ", quoting=3, header=None, index_col=0,skiprows=1)

**LexVec Word Embedding**

In [ ]:
embedding_matrix = df_embedding.to_numpy()

**Extracting Vocabulary from word embedding**

In [ ]:
vocab = []

for word in list(df_embedding.index):
  vocab.append(str(word))

In [ ]:
vocab_size , vocab_dim = embedding_matrix.shape
vocab_size, vocab_dim

**Reading the data**


In [ ]:
traindata = pd.read_csv("/content/drive/MyDrive/dataset/split_dataset/train.csv",sep=',')
testdata = pd.read_csv("/content/drive/MyDrive/dataset/split_dataset/test.csv",sep=',')
validdata = pd.read_csv("/content/drive/MyDrive/dataset/split_dataset/valid.csv",sep=',')

In [ ]:
print(traindata.shape)
print(testdata.shape)
print(validdata.shape)

In [ ]:
X_train_data = traindata['tweets']
y_train_labels = traindata['labels']
X_test_data = testdata['tweets']
y_test_labels = testdata['labels']
X_valid_data = validdata['tweets']
y_valid_labels  = validdata['labels']

In [ ]:
y_test_labels

In [ ]:
max_input_length = get_max_len(X_train_data)
mean_input_length = get_mean_len(X_train_data)
max_input_length, int(mean_input_length)



**Tokenization**

In [ ]:
tokenizer = Tokenizer(num_words= vocab_size)
tokenizer.fit_on_texts(vocab)

maxlen = max_input_length

# tokenizing training data
sequences_train = tokenizer.texts_to_sequences(X_train_data)
X_train_data = pad_sequences(sequences_train, maxlen=maxlen,padding='post',truncating='post')

# # tokenizing test data
sequences_test = tokenizer.texts_to_sequences(X_test_data)
X_test_data = pad_sequences(sequences_test, maxlen=maxlen,padding='post',truncating='post')

# tokenize the validation data
sequences_valid = tokenizer.texts_to_sequences(X_valid_data)
X_valid_data = pad_sequences(sequences_valid, maxlen=maxlen,padding='post',truncating='post')


In [ ]:
print(sequences_train[0])
print(X_train_data[0])
tokenizer.sequences_to_texts([sequences_train[0]])

In [ ]:
checkpoint_path = "/content/drive/MyDrive/models/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
'''class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    try:
      if(logs.get('accuracy')>0.979):
        self.model.stop_training = True
    except:
      if(logs.get('acc')>0.979):
        self.model.stop_training = True


acc_callback = myCallback()
early_stopping_callback = tf.keras.callbacks.EarlyStopping()
'''

def scheduler(epoch, lr):
  if epoch < 5:
    print("Learning rate", lr)
    return lr
  else:
    lr = lr * tf.math.exp(-0.1)
    print("Epoch: {} Learning rate: {}", epoch, lr)
    return lr

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
class SelfMatchingLayer(tf.keras.layers.Layer):
    def __init__(self,  seq_length, embed_dim, **kwargs):
      self.seq_length = seq_length
      self.embed_dim  = embed_dim
      self.output_dim = seq_length*embed_dim # dim of the paramter W
      super(SelfMatchingLayer, self).__init__(**kwargs)

    def build(self, input_shape): # input_shape is the shape of x
      self.M = self.add_weight(name='M', # M is square matrix of shape n*k X n*k , see the paper selfmatching network
                                      shape=tf.TensorShape((self.output_dim, self.output_dim )).as_list(),
                                      initializer='glorot_uniform',
                                      trainable=True)
      super(SelfMatchingLayer, self).build(input_shape)  # Be sure to call this at the end
      
    def call(self,x):
      # print(K.int_shape(x))
      # create an identity matrix of shape seq_length
      t = tf.eye(self.seq_length, dtype=x.dtype) 
      # Add one dimension to x and one dimension to t
      xt = tf.expand_dims(x, 1) * tf.expand_dims(t, 2)
      # Reshape, output is basically a diagnoal magtrix whose entries are embeddings
      output = tf.reshape(xt, (-1, self.seq_length, self.output_dim))
      # E*M*E^T happens bellow
      tmp = tf.matmul(output, self.M)
      result = tf.matmul(tmp,output, transpose_b=True)
      # maximize by rows
      result = tf.reduce_max(result, axis=-1)
      # take softmax
      result = tf.nn.softmax(result)
      result = tf.einsum('ij,ijk->ik',result,x)
      return result
    def compute_output_shape(input_shape):
      return (input_shape[0], self.output_dim)

In [ ]:
input_layer = tf.keras.Input(shape=(maxlen, ),name='text_input')
input_embedding_layer = tf.keras.layers.Embedding(vocab_size, vocab_dim,
                        embeddings_initializer=Constant(embedding_matrix), 
                        trainable = False)
bi_lstm_layer = layers.Bidirectional(layers.LSTM(100, dropout=0.4,
                        recurrent_dropout=0.4))
selfmatching_layer = SelfMatchingLayer(maxlen,vocab_dim, name='selfmatch')

In [ ]:
embedding_output = input_embedding_layer(input_layer)

selfmatch_output = selfmatching_layer(embedding_output)
bi_lstm_output = bi_lstm_layer(embedding_output)

concat_features = tf.concat([selfmatch_output, bi_lstm_output],-1)
sarc_prediction = layers.Dense(64,activation='relu',name='activation_relu')(concat_features)
sarc_prediction = layers.Dense(2,activation='softmax',name='activation_softmax')(sarc_prediction)
#sarc_prediction = layers.Softmax()(sarc_prediction)

In [ ]:
model = tf.keras.Model(input_layer, sarc_prediction)
tf.keras.utils.plot_model(model,'model.jpg',show_shapes=True)

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
              optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(np.array(X_train_data), np.array(y_train_labels), validation_data=(np.array(X_valid_data), np.array(y_valid_labels)), epochs=20, callbacks= [callback]) 

In [ ]:
prediction = model.predict(np.array(X_test_data))

In [ ]:
prediction[11]

In [ ]:
y_test_pred = np.argmax(prediction,axis=1)

In [ ]:
y_test_pred

In [ ]:
np.array(y_test_labels)

In [ ]:
print("precision_score", precision_score(y_test_labels, y_test_pred, average='micro'))

In [ ]:
print("recall_score", recall_score(y_test_labels, y_test_pred, average='micro'))

In [ ]:
print("f1_score_micro", f1_score(y_test_labels, y_test_pred, average='micro'))

In [ ]:
print("f1_score_binary", f1_score(y_test_labels, y_test_pred, average='binary'))

In [ ]:
print("accuracy_score", accuracy_score(y_test_labels, y_test_pred))

In [ ]:
print("avg_precison_score", precision_score(y_test_labels, y_test_pred, average='micro'))

#Old data training

In [ ]:
history = model.fit(np.array(X_train_data), np.array(y_train_labels), validation_split=0.2, epochs=10,callbacks= [cp_callback]) 

In [ ]:
prediction = model.predict(np.array(X_test_data))
y_test_pred = np.argmax(prediction,axis=1)
print("precision_score", precision_score(y_test_labels, y_test_pred, average='micro'))
print("recall_score", recall_score(y_test_labels, y_test_pred, average='micro'))
print("f1_score_micro", f1_score(y_test_labels, y_test_pred, average='binary'))
print("accuracy_score", accuracy_score(y_test_labels, y_test_pred))

print(y_test_pred, y_test_labels)

In [ ]:
test = [
        "TensorFlow Datasets is a collection of datasets ready to use with TensorFlow",
        "It can be difficult for a beginner to the field of deep learning to know what type of network to use. There are so many types of networks to choose from and new methods being published and discussed every day.",
        "It can be difficult for a beginner to the field of deep learning to know what type of network to use.",
        "There are so many types of networks to choose from and new methods being published and discussed every day."
        "I'm glad we're having a rehearsal dinner",
        "I rarely practice my meals before I eat",
        "I'm glad we're having a rehearsal dinner. I rarely practice my meals before I eat",
        "Thank you for explaining that my eye cancer isn't going to make me deaf",
        "I feel so fortunate that an intellectual giant like yourself would deign to operate on me",
        "Thank you for explaining that my eye cancer isn't going to make me deaf. I feel so fortunate that an intellectual giant like yourself would deign to operate on me."]
seq_test = tokenizer.texts_to_sequences(test)
test = pad_sequences(seq_test, maxlen=maxlen,truncating='post', padding='post')

In [ ]:
# del model
# # Recreate the exact same model purely from the file:
# model = keras.models.load_model('path_to_my_model')

In [ ]:
np.argmax(model.predict([test]),axis=1)

In [ ]:
try:
  plt.plot(history.history['acc'], label='accuracy')
  plt.plot(history.history['val_acc'], label = 'val_accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.ylim([0.5, 1])
  plt.legend(loc='lower right')
  plt.savefig('plot.png')
except:
  plt.plot(history.history['accuracy'], label='accuracy')
  plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.ylim([0.5, 1])
  plt.legend(loc='lower right')
  plt.savefig('plot.png')

In [ ]:
model.save('self-match-bi-lstm')